In [1]:
from dotenv import load_dotenv
_ = load_dotenv("config.env")

from common.mongodb import *
from common.files import *


drop_collection=True
collection = get_mongo_collection("filter_perfomance_test", drop_collection=drop_collection)

# new mock data
if drop_collection:
    insert_into_mongo(collection, 1000000)

def exec_query(query, collection):
    t1 = datetime.datetime.now()
    query_result = collection.find(query)
    list_query_result = list(query_result)
    len_list = len(list_query_result)
    t2 = datetime.datetime.now()
    dt = round(((t2-t1).total_seconds()*1000), 2)
    print(f"Query took {dt} ms and has {len_list} results.")

    return list_query_result

collection.find_one()

2025-01-31 19:46:02,676 - my_logger - INFO - host: localhost


host: localhost


{'_id': '8562c390-dfe7-4262-bfc1-a0e3a9fa8f68',
 'filepath': 'fp0_2/fp1_5/fp2_7/File_2109.docx',
 'filepath_custom': 'fp_c0_3/fp_c1_2/fp_c2_4/File_2109.docx',
 'category': 'Rechnung',
 'src': {'type': 'local', 'fp': None},
 'metadata': {'infos': {'receiptid': 'rechnung 09598026-b2c6-4bd3-9e9d-ee4c56cbd394',
   'receiptdate': datetime.datetime(2024, 6, 19, 9, 39, 57, 144000)},
  'category': {'maincategory': 'concern', 'subcategory': 'hope'},
  'buyer': {'firstname': 'Stephanie',
   'lastname': 'Hernandez',
   'company': 'Price-Skinner'},
  'seller': {'firstname': 'Jessica',
   'lastname': 'Wallace',
   'company': 'Ortiz Ltd'},
  'price': {'price': 926.69}},
 'summary': 'Small summary of the file ... .'}

In [ ]:
# normal filter
# many results
query = { "metadata.infos.receiptid": {"$regex": "^rechnung"}}
exec_query(query, collection)

# only few results
query = { "metadata.infos.receiptid": {"$regex": "^rechnung 2b8"}}
exec_query(query, collection)

# no results
query = { "metadata.infos.dummyreceiptid": {"$regex": "^rechnung"}}
exec_query(query, collection)

# show no caching
query = { "metadata.infos.receiptid": {"$regex": "^rechnung"}}
exec_query(query, collection)

In [ ]:
# numbers
query = {"metadata.price.price": {"$gt": 984}}
exec_query(query, collection)

In [ ]:
# dates

query = {
    "$expr": {"$lt": ["$metadata.infos.receiptdate", datetime.datetime(9999, 1, 1)]}
}
exec_query(query, collection)

query = {"$expr": {"$eq": [{"$month": "$metadata.infos.receiptdate"}, 10]}}
exec_query(query, collection)

In [ ]:
# and/or

query = {
    "$and": [
        {"metadata.infos.receiptid": {"$regex": "^rechnung"}},
        {"$expr": {"$eq": [{"$month": "$metadata.infos.receiptdate"}, 10]}},
        {
            "$or": [
                {"metadata.price.price": {"$lt": 8200}},
            ]
        },
    ]
}

exec_query(query, collection)

In [2]:
# compound index
collection.create_index([
    ("metadata.infos.receiptid", "text"),
    ("metadata.category.filepath_custom", "text")
])


OperationFailure: An equivalent index already exists with a different name and options. Requested index: { v: 2, key: { _fts: "text", _ftsx: 1 }, name: "metadata.infos.receiptid_text_metadata.category.filepath_custom_text", weights: { metadata.category.filepath_custom: 1, metadata.infos.receiptid: 1 }, default_language: "english", language_override: "language", textIndexVersion: 3 }, existing index: { v: 2, key: { _fts: "text", _ftsx: 1 }, name: "metadata.infos.receiptid_text_metadata.category.maincategory_text", weights: { metadata.category.maincategory: 1, metadata.infos.receiptid: 1 }, default_language: "english", language_override: "language", textIndexVersion: 3 }, full error: {'ok': 0.0, 'errmsg': 'An equivalent index already exists with a different name and options. Requested index: { v: 2, key: { _fts: "text", _ftsx: 1 }, name: "metadata.infos.receiptid_text_metadata.category.filepath_custom_text", weights: { metadata.category.filepath_custom: 1, metadata.infos.receiptid: 1 }, 

In [ ]:
query = {"$text": {"$search": "fp_c0_3/fp_c1_4/fp_c2_4/File_6504.do"}}
exec_query(query, collection)